## Домашнее задание 2

Задание 1

In [1]:
#Создаем соединение с SQL (+создаем ввод пароля)
import psycopg2
import getpass

conn = psycopg2.connect(user='postgres', 
                        database='postgres', 
                        host='localhost', 
                        port='5433', 
                        password = getpass.getpass('Insert password: ')
                       )
print(conn.get_dsn_parameters())

Insert password: ········
{'user': 'postgres', 'dbname': 'postgres', 'host': 'localhost', 'port': '5433', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'}


Задание 2

In [2]:
# Получение данных
#Нам необходимы следующие данные: ISIN, тип купона (постоянный и дисконтный), столбец, указывающий, является ли 
#облигация амортизационной, купон, количество купонов в год, цена и дата обратного выкупа по оферете,
#кколичество выплаченных купонов, номинальная стоимость, дата погашения облигации, дата выпуска облигации (как дата начала торгов)
#Из всего пречисленного нас особо интересуют непогашенные облигации, также убираем отсутствующие значения ISIN
#Поскольку по облигации может быть разное значение купона, отбираем только максимальное его значение, также берем 
#максимальное значение buybackdate и buybackprice для отбора только одного для каждой облигации
import pandas as pd
query = """SELECT "ISIN", "IssuerName", "CouponType", "AmortisedMty", "CouponPerYear", "NumCoupons",
            "CPN", "BUYBACKDATE", "BUYBACKPRICE", "FaceValue", "EndMtyDate", "CPN_PERIOD", "FirstDate", "HaveOffer"
            FROM (
				SELECT DISTINCT public.bond_description."ISIN, RegCode, NRDCode", public.bond_description."IssuerName", public.bond_description."CouponType", 
				public.bond_description."AmortisedMty", public.bond_description."CouponPerYear", public.bond_description."NumCoupons",
				public.bond_description."FaceValue", public.bond_description."EndMtyDate", "HaveOffer"
				FROM public.bond_description INNER JOIN public.quotes
				ON public.quotes."ISIN" = public.bond_description."ISIN, RegCode, NRDCode"  
				WHERE "ISIN, RegCode, NRDCode" IS NOT NULL AND "Status" != 'Погашен') AS S1
			INNER JOIN (SELECT "ISIN", MAX("CPN_PERIOD") AS "CPN_PERIOD", MAX("CPN") AS "CPN", MAX("BUYBACKDATE") AS "BUYBACKDATE", MAX("BUYBACKPRICE") AS "BUYBACKPRICE", MIN("TIME") AS "FirstDate" 
                FROM public.quotes GROUP BY "ISIN") AS S2
			ON S1."ISIN, RegCode, NRDCode"=S2."ISIN";"""
df = pd.read_sql_query(query, conn) 
df['EndMtyDate'] = pd.to_datetime(df['EndMtyDate'])
df.BUYBACKPRICE = df.BUYBACKPRICE.fillna(0) 
df

,ISIN,IssuerName,CouponType,AmortisedMty,CouponPerYear,NumCoupons,CPN,BUYBACKDATE,BUYBACKPRICE,FaceValue,EndMtyDate,CPN_PERIOD,FirstDate,HaveOffer
0,RU000A0JTR15,Ростелеком,Переменный,True,2,10,40.3900,9999-05-05,100.0,1000.00,2018-03-06,182.0,2013-03-22,True
1,RU000A0JULC6,РОСБАНК,Фиксированный,False,4,40,0.0300,None,0.0,1000.00,2024-04-30,92.0,2014-04-30,False
2,RU000A0JVCK6,Первое КоллекторБюро,Постоянный,True,4,12,42.3836,None,0.0,1000.00,2018-04-19,91.0,2015-04-23,False
3,RU000A0JURL4,ВТБ Лизинг Финанс,Переменный,True,4,40,34.2800,9999-05-05,100.0,1000.00,2024-07-17,91.0,2014-07-30,True
4,RU000A0JVPQ5,ПИК ГК,Переменный,False,2,20,71.0548,9999-05-05,100.0,1000.00,2025-08-08,182.0,2015-08-21,True
5,RU000A0JTW34,Вертолеты России,Переменный,False,2,10,41.1400,9999-05-05,100.0,1000.00,2018-04-20,182.0,2013-05-29,True
6,RU000A0JTU85,РЖД,Переменный,False,2,30,40.8900,9999-05-05,100.0,1000.00,2028-03-21,182.0,2013-04-12,True
7,RU000A0JUSH0,ЮниКредит Банк,Переменный,False,2,10,52.3600,9999-05-05,100.0,1000.00,2019-08-06,182.0,2014-08-13,True
8,RU000A0JU146,Восточный Экспресс КБ,Переменный,False,2,10,92.2500,9999-05-05,100.0,1000.00,2018-07-17,182.0,2013-07-23,True
9,RU000A0JUW31,Россельхозбанк,Переменный,False,4,40,27.6740,9999-05-05,100.0,1000.00,2024-09-27,91.0,2014-10-10,True


##### Задание 3

In [3]:
#Используем следуюшие аргументы: ISIN, дата, логин
def my_fun(ISIN, RegCode, NRDCode, date, db_login):
    conn=psycopg2.connect(user=db_login['user'],
                      database=db_login['database'],
                      host=db_login['host'],
                      port=db_login['port'],
                      password=getpass.getpass('Insert password: '))

In [4]:
#Здесь приведена попытка написать код для неамортизационных облигаций с постоянным купоном. 
#подозреваю, что нужно использовать еще pivot для вывода таблицы


import pandas as pd
import numpy as np
import datetime

df['EndMtyDate'] = pd.to_datetime(df['EndMtyDate'])
ISIN = df['ISIN']
coup = df['CPN']
coup_per_year = df['CouponPerYear']
coup_num = df['NumCoupons']
df['FirstDate'] = pd.to_datetime(df['FirstDate'])
buyback_date = df['BUYBACKDATE']
date = df['FirstDate']
buyback_price = df['BUYBACKPRICE']
date = pd.to_datetime(date)
AmortisedMty = df.AmortisedMty.astype(str)
CouponType = df.CouponType.astype(str)
payment = coup
df.CPN_PERIOD = pd.to_timedelta(df.CPN_PERIOD)
for i in range (0,len(df)): 
    if df.AmortisedMty.values[i] == False and df.CouponType.values[i] == 'Постоянный':
        if df.BUYBACKDATE.values[i] is None:
            while date.values[i] < df.EndMtyDate.values[i]:
                payment.values[i] = coup.values[i]
                df.CPN_PERIOD = pd.to_timedelta(df.CPN_PERIOD)
                date.values[i] = date.values[i] + df.CPN_PERIOD.values[i]
                if date.values[i] == df.EndMtyDate.values[i]:
                    payment.values[i] = df.CPN.values[i] + df.FaceValue.values[i]
                    date.values[i] = df.EndMtyDate.values[i] 
        if df.BUYBACKDATE.values[i] is not None:
            while (date.values[i] < df.BUYBACKDATE.values[i]) and (df.BUYBACKDATE.values[i] < df.EndMtyDate.values[i]):
                payment.values[i] = df.CPN.values[i]
                date.values[i] = date.values[i] + df.CPN_PERIOD.values[i]
                if date.values[i] == df.BUYBACKDATE.values[i]:
                    payment.values[i] = df.CPN.values[i] + df.BUYBACKPRICE.values[i]
                    date.values[i] = df.BUYBACKDATE.values[i] 
    print('Step :'+str(i))

payment
# return(payment)
# return(payment)
#if df.AmortisedMty.values[i] == True and df.CouponType.values[i] == 'Постоянный':
      # if df.BUYBACKDATE.values[i] is None:
        
    
      # if df.BUYBACKDATE.values[i] is not None:    
            

#if df.CouponType.values[i] == 'Дисконт':
     # if df.BUYBACKDATE.values[i] is None:
        
    
     # if df.BUYBACKDATE.values[i] is not None:    

Step :0
Step :1
Step :2
Step :3
Step :4
Step :5
Step :6
Step :7
Step :8
Step :9
Step :10


KeyboardInterrupt: 

## Код Марины (для просмотра дз 4 пролистайте ниже)

In [4]:
# import packages

import psycopg2
import getpass
import numpy as np
import datetime
import time
import pandas as pd
import dateutil.relativedelta as dr

# connect with database

conn = psycopg2.connect(user='postgres', 
                        database='postgres', 
                        host='localhost', 
                        port='5433', 
                        password = getpass.getpass('Insert password: ')
                       )
# check parameters

print(conn.get_dsn_parameters())

Insert password: ········
{'user': 'postgres', 'dbname': 'postgres', 'host': 'localhost', 'port': '5433', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'}


In [6]:
# extract data from bond_description and quotes

query = """SELECT *
            FROM (SELECT MAX("CPN") AS "CPN", "ISIN", max("BUYBACKDATE") AS "BUYBACKDATE",
                  max("BUYBACKPRICE") AS "BUYBACKPRICE"
                  FROM public.quotes  
                  GROUP BY "ISIN") AS a
            INNER JOIN(SELECT "ISINCode", "FaceValue", "HaveOffer", "AmortisedMty", 
                       "CouponPerYear", "NumCoupons", "EndMtyDate", "CouponType", "IssuerName"
                       FROM public.bond_description 
                       WHERE "Status"!='Погашен') AS b
            ON a."ISIN" = b."ISINCode";"""

# read query in python

data = pd.read_sql_query(query, conn) 

# Extract more specified data 
data.drop(['ISINCode'], axis=1)
data.BUYBACKPRICE = data.BUYBACKPRICE.fillna(0)
data = data[~data.CPN.isna()].reset_index(drop = True)
data = data[data.CPN != 0]
data = data[data.CouponPerYear > 0]

# if you had timestamp format in any of extracted fields so change format to date
# in my case it was only buybackdate

#data.BUYBACKDATE = data.BUYBACKDATE.dt.date
data.head()

,CPN,ISIN,BUYBACKDATE,BUYBACKPRICE,ISINCode,FaceValue,HaveOffer,AmortisedMty,CouponPerYear,NumCoupons,EndMtyDate,CouponType,IssuerName
0,97.50,RU000A0JRYJ0,None,0.0,RU000A0JRYJ0,1000.0,False,False,2,9,2016-11-29,Постоянный,Автодор ГК
1,35916.00,RU000A0JTG34,None,0.0,RU000A0JTG34,1000.0,False,True,4,20,2017-12-17,Постоянный,Воронежская Обл Прав
2,43515.00,RU000A0JU823,None,0.0,RU000A0JU823,1000.0,False,True,4,20,2018-10-23,Фиксированный,Воронежская Обл Прав
3,29.29,RU000A0JUWK6,None,0.0,RU000A0JUWK6,1000.0,False,True,4,20,2019-10-14,Постоянный,Воронежская Обл Прав
4,26.55,RU000A0JS1E8,None,0.0,RU000A0JS1E8,1000.0,False,True,4,20,2016-12-14,Постоянный,Вологодская Обл Прав


In [7]:
# function to count coupon payments

def coupon_count(coup_str):
    full_matrix = pd.DataFrame() # local var
    end_date = coup_str.BUYBACKDATE.values[0] if coup_str.BUYBACKDATE.values[0] is not None else coup_str.EndMtyDate.values[0]
    a = int(coup_str.CouponPerYear.values[0])
    b = float(coup_str.NumCoupons.values[0])
    types = True
    if coup_str.CPN is not None or coup_str.CPN != 0:
        if coup_str.AmortisedMty.astype(str).values[0] == 'False':
            types = False
            amount = coup_str.CPN.values[0]
            for j in np.linspace((b - 1) / a, 0, b):
                date = coup_str.EndMtyDate.values[0] - dr.relativedelta(days = round(365.25 * j))
                if date >= end_date and coup_str.EndMtyDate.values[0] != end_date :
                    date = end_date
                    amount = coup_str.BUYBACKPRICE.values[0]
                    full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                    break
                elif date >= end_date and coup_str.EndMtyDate.values[0] == end_date:
                    date = end_date
                    amount += coup_str.FaceValue.values[0]
                    full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                    break
                full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                amount = coup_str.CPN.values[0]
        else:
            percent = float(coup_str.CPN.values[0]) / float(coup_str.FaceValue.values[0])
            index = (b - 1)
            face = coup_str.FaceValue.values[0]
            while index >= 0:
                amount = percent * face
                date = coup_str.EndMtyDate.values[0] - dr.relativedelta(days = round(365.25*index/a))
                if date >= end_date and coup_str.EndMtyDate.values[0] != end_date :
                    date = end_date
                    amount = coup_str.BUYBACKPRICE.values[0]
                    full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                    break
                elif date >= end_date and coup_str.EndMtyDate.values[0] == end_date:
                    date = end_date
                    amount = face
                    full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                    break
                full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                index -= 1
                face = face * (1 - percent)
    my_index = pd.MultiIndex.from_tuples([(coup_str.ISIN.values[0], 'Amortised' if types else 'Not Amortised'
                                           , coup_str.IssuerName.values[0] )], names=['ISIN', 'Amortisation', 'Issuer'])
    full_matrix = pd.DataFrame(full_matrix.values, index = my_index, columns = full_matrix.columns)
    return full_matrix

In [8]:
# full_matrix contains coupon payments for all the emitents
# global var:

full_matrix = pd.DataFrame()

# fill in full_matrix

for i in data.iterrows():
    full_matrix = pd.concat([full_matrix, coupon_count(pd.DataFrame(i[1]).T)], sort = True)
    
# example
full_matrix.fillna(0).head(20)

/Users/diana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  del sys.path[0]


,,,2006-08-19,2006-10-06,2007-01-05,2007-02-18,2007-04-06,2007-06-11,2007-06-15,2007-06-16,2007-07-07,2007-08-11,...,2041-10-28,2041-11-19,2042-04-29,2042-05-21,2042-10-28,2042-11-19,2043-04-29,2043-05-21,2043-11-19,2044-05-20
ISIN,Amortisation,Issuer,,,,,,,,,,,,,,,,,,,,,
RU000A0JRYJ0,Not Amortised,Автодор ГК,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JTG34,Amortised,Воронежская Обл Прав,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JU823,Amortised,Воронежская Обл Прав,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JUWK6,Amortised,Воронежская Обл Прав,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JS1E8,Amortised,Вологодская Обл Прав,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JTF68,Amortised,Вологодская Обл Прав,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JV3M2,Amortised,Волжский Адм,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JRGR0,Amortised,Волгоградская Обл Адм,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JUP89,Amortised,Волгоградская Обл Адм,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# check
print(full_matrix.values[0].T != 'nan')
print(full_matrix.shape) # table: 500 rows, 2230 columns

True
(500, 3658)


/Users/diana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


In [10]:
# from the given date to the end
import datetime
from datetime import datetime, timedelta, date
def from_date_matrix(issuer_name, date):
    
    spec_payments = pd.DataFrame()
    for i in range(0, len(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).columns)):
        date_size = int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, \
        how = 'all').fillna(0).columns[i].split('-')[0]) * 365 + int(full_matrix.xs(str(issuer_name), \
        level = 2).dropna(axis = 1, how = 'all').fillna(0).columns[i].split('-')[1]) * 365 / 12 + \
        int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).columns[i].split('-')[2])
        if int(date.split('-')[0]) * 365 + int(date.split('-')[1]) * 365 / 12 + int(date.split('-')[2]) < date_size:
            data = full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).iloc[:, i:i+1]
            spec_payments = pd.concat([spec_payments, data], axis = 1)
    return spec_payments

In [11]:
# example
# insert date in format yyyy-mm-dd
from_date_matrix('Банк ВТБ','2010-01-01')

,,2010-01-05,2010-04-06,2010-07-06,2010-10-06,2011-01-05,2011-04-06,2011-07-07,2011-10-06,2012-01-05,2012-04-06,...,2023-03-30,2023-06-14,2023-06-30,2023-09-13,2023-09-29,2023-12-14,2024-03-14,2024-06-13,2024-09-13,2024-12-13
ISIN,Amortisation,,,,,,,,,,,,,,,,,,,,,
RU000A0JTK20,Not Amortised,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0000,0.00,0.0000,0.00,0.0000,0.0000,0.0000,0.0000,0.0000
RU000A0JTN01,Not Amortised,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0000,0.00,0.0000,0.00,0.0000,0.0000,0.0000,0.0000,0.0000
RU000A0JNGU3,Not Amortised,43726.0,43726.0,43726.0,43726.0,43726.0,43726.0,43726.0,43726.0,43726.0,43726.0,...,0.00,0.0000,0.00,0.0000,0.00,0.0000,0.0000,0.0000,0.0000,0.0000
RU000A0JUQE1,Not Amortised,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0000,0.00,0.0000,0.00,0.0000,0.0000,0.0000,0.0000,0.0000
RU000A0JU773,Not Amortised,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,35.28,0.0000,35.28,0.0000,35.28,0.0000,0.0000,0.0000,0.0000,0.0000
RU000A0JV3Q3,Not Amortised,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,42.3836,0.00,42.3836,0.00,42.3836,42.3836,42.3836,42.3836,42.3836


In [12]:
# payments on specified date
def on_spec_date(issuer_name, date):
    on_spec_date_payments = pd.DataFrame()
    for i in range(0,len(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).columns)):
        date_size = int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, \
        how = 'all').fillna(0).columns[i].split('-')[0]) * 365 + int(full_matrix.xs(str(issuer_name),\
        level = 2).dropna(axis = 1, how = 'all').fillna(0).columns[i].split('-')[1]) * 365 / 12 + \
        int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how='all').fillna(0).columns[i].split('-')[2])
        if int(date.split('-')[0]) * 365 + int(date.split('-')[1]) * 365 / 12 + int(date.split('-')[2]) == date_size:
            data = full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).iloc[:, i:i+1]
            on_spec_date_payments = pd.concat([on_spec_date_payments, data], axis = 1)
    return on_spec_date_payments

In [13]:
on_spec_date('Банк ВТБ','2016-02-16')

,,2016-02-16
ISIN,Amortisation,
RU000A0JTK20,Not Amortised,0.0
RU000A0JTN01,Not Amortised,44665.0
RU000A0JNGU3,Not Amortised,0.0
RU000A0JUQE1,Not Amortised,0.0
RU000A0JU773,Not Amortised,0.0
RU000A0JV3Q3,Not Amortised,0.0


In [38]:
# from the beginning date to today

import datetime
acctual_date = datetime.datetime.now().strftime("%Y-%m-%d")

def from_date_to_today(issuer_name):
    from_date_to_today_payments = pd.DataFrame()
    for i in range(0, len(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).columns)):
        date_size = int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, \
        how = 'all').fillna(0).columns[i].split('-')[0]) * 365 + \
        int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).columns[i].split('-')[1]) * 365/12 \
        + int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).columns[i].split('-')[2])
        if int(datetime.datetime.now().strftime("%Y-%m-%d").split('-')[0]) * 365 + \
        int(datetime.datetime.now().strftime("%Y-%m-%d").split('-')[1]) * 365/12 + \
        int(datetime.datetime.now().strftime("%Y-%m-%d").split('-')[2]) > date_size:
            data = full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).iloc[:, i:i+1]
            from_date_to_today_payments = pd.concat([from_date_to_today_payments, data], axis = 1)
    return from_date_to_today_payments

In [15]:
from_date_to_today('Россельхозбанк')

,,2007-08-11,2008-02-10,2008-03-28,2008-08-10,2008-09-27,2008-12-04,2009-02-09,2009-03-28,2009-05-28,2009-06-05,...,2019-10-11,2019-10-13,2019-10-15,2019-10-27,2019-10-30,2019-11-10,2019-11-14,2019-12-14,2019-12-28,2019-12-29
ISIN,Amortisation,,,,,,,,,,,,,,,,,,,,,
RU000A0JP2G4,Not Amortised,46.12,46.12,0.0000,46.12,0.0000,0.00,46.12,0.0000,0.00,0.00,...,0.0000,0.0000,0.0000,0.0000,0.0,0.00,0.0000,0.0000,0.000,0.0000
RU000A0JUPQ7,Not Amortised,0.00,0.00,0.0000,0.00,0.0000,0.00,0.00,0.0000,0.00,0.00,...,0.0000,0.0000,0.0000,0.0000,0.0,0.00,0.0000,0.0000,0.000,0.0000
RU000A0JUQ05,Not Amortised,0.00,0.00,0.0000,0.00,0.0000,0.00,0.00,0.0000,0.00,0.00,...,0.0000,0.0000,0.0000,0.0000,0.0,0.00,0.0000,0.0000,0.000,0.0000
RU000A0JPF47,Not Amortised,0.00,0.00,61.5808,0.00,61.5808,0.00,0.00,61.5808,0.00,0.00,...,0.0000,0.0000,0.0000,0.0000,0.0,0.00,0.0000,0.0000,0.000,0.0000
RU000A0JPMT8,Not Amortised,0.00,0.00,0.0000,59.34,0.0000,0.00,59.34,0.0000,0.00,0.00,...,0.0000,0.0000,0.0000,0.0000,0.0,0.00,0.0000,0.0000,0.000,0.0000
RU000A0JPTE5,Not Amortised,0.00,0.00,0.0000,0.00,0.0000,34.41,0.00,0.0000,0.00,34.41,...,0.0000,0.0000,0.0000,0.0000,0.0,0.00,0.0000,0.0000,0.000,0.0000
RU000A0JQ1H3,Not Amortised,0.00,0.00,0.0000,0.00,0.0000,0.00,0.00,0.0000,67.32,0.00,...,0.0000,0.0000,0.0000,0.0000,0.0,0.00,0.0000,0.0000,0.000,0.0000
RU000A0JQK64,Not Amortised,0.00,0.00,0.0000,0.00,0.0000,0.00,0.00,0.0000,0.00,0.00,...,0.0000,0.0000,0.0000,0.0000,0.0,0.00,64.8219,0.0000,0.000,0.0000
RU000A0JQK72,Not Amortised,0.00,0.00,0.0000,0.00,0.0000,0.00,0.00,0.0000,0.00,0.00,...,0.0000,0.0000,0.0000,0.0000,0.0,0.00,64.8219,0.0000,0.000,0.0000


# Домашнее задание 4 (это уже моя часть)

In [ ]:
#Сначала проводим установку всего необходимого

In [10]:
pip install --upgrade pip setuptools

Requirement already up-to-date: pip in ./anaconda3/lib/python3.7/site-packages (19.3.1)
Requirement already up-to-date: setuptools in ./anaconda3/lib/python3.7/site-packages (42.0.2)
Note: you may need to restart the kernel to use updated packages.


In [9]:
conda install -c conda-forge weasyprint 

Solving environment: done

## Package Plan ##

  environment location: /Users/diana/anaconda3

  added / updated specs:
    - weasyprint


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py37_0         148 KB  conda-forge
    conda-4.8.0                |           py37_1         3.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following packages will be UPDATED:

  ca-certificates    anaconda::ca-certificates-2019.11.27-0 --> conda-forge::ca-certificates-2019.11.28-hecc5488_0

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi                                          anaconda --> conda-forge
  conda                                            anaconda --> conda-forge
  openssl                anaconda::openssl-1.1.1

In [8]:
conda install -c anaconda jinja2

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [5]:
conda install -c conda-forge gtk2
#Здесь я пошла немного другим путем и не загружала никаких программ (но, кажется, все работает)

Solving environment: done

## Package Plan ##

  environment location: /Users/diana/anaconda3

  added / updated specs:
    - gtk2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    atk-2.32.0                 |       hbe7ca29_0         348 KB  conda-forge
    cairo-1.16.0               |    h0ab9d94_1001         1.3 MB  conda-forge
    cairocffi-1.1.0            |             py_0          67 KB  conda-forge
    gdk-pixbuf-2.36.12         |    h284f8de_1003         568 KB  conda-forge
    glib-2.58.3                |    h9d45998_1002         3.1 MB  conda-forge
    gobject-introspection-1.58.2|py37h93883a9_1002         1.2 MB  conda-forge
    gtk2-2.24.32               |       hf4cab1c_0         7.0 MB  conda-forge
    harfbuzz-2.4.0             |       h92b87b8_1         1.2 MB  conda-forge
    jpeg-9c                    |    h1de35cc_1001         237 KB  conda-forge
    pango-1.42.4    

In [51]:
#С картинкой не делала, поскольку нет примера, на котором это можно попробовать (поэтому везде эти '____')
#С таблицей все получилось 
from __future__ import print_function
from jinja2 import Environment, FileSystemLoader
from weasyprint import HTML, Document
import os


env = Environment(loader=FileSystemLoader('.'))
template = env.get_template("myreport.html") 
template_vars = {"title": "Отчет составлен Муталовой Дианой Рустемовной",
                "INTRO": "Данный отчет содержит таблицу платежей эмитента Банк ВТВ на дату 2016-02-16.", 
                 "TABLE_DESC": "В таблице представлена таблица платежей облигации Банка ВТБ", 
                 "TABLE": on_spec_date('Банк ВТБ','2016-02-16').to_html(), 
                 "IMAGE_DESC": "На графике изображено _____", 
                 "IMAGE": os.path.realpath('_______.png')
                }
# os.path.realpath - путь к файлу относительно рабочей папки. Код выше подразумевает, что картинка хранится в рабочей папке.
# Собираем и печатаем репорт
html_out = template.render(template_vars, presentational_hints=True)
HTML(string=html_out).write_pdf('my_report.pdf', presentational_hints=True)